# Group Members:

*   Name 1
*   Name 2
*   Name 3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd ""

In [ ]:
!pwd

# Import required libraries

In [ ]:
import numpy as np

# 1. Linear regression

## 1.1 Computing the minimum via the normal equations

#### 1 Complete the function polyfit_inv_normal_eq for computing w&ast;

  $$w^* = (\Phi^{T}\Phi)^{-1}\Phi^{T}y$$




**1) Calculate Phi**

$$\Phi = \begin{bmatrix} 
1 & x_{1} & x_{1}^{2} & x_{1}^{3}\\
1 & x_{2} & x_{2}^{2} & x_{2}^{3}\\
\vdots & \vdots & \vdots & \vdots\\
1 & x_{m} & x_{m}^{2} & x_{m}^{3}
\end{bmatrix}$$

In [ ]:
#TOY EXAMPLE

# Define x --> (m, 1)

x = np.matrix([range(2,5)]).T


# Define the number of coefficients to calculate (components of w --> w = (w0, w1, ..., wn))
num_coefficients = 4

# Calculate phi
phi = np.matrix([])

In [ ]:
print("Independent variable: \n",x,"\n\nDesign matrix:\n",phi)

**2) Calculate w&ast;**

In [ ]:
# Define y --> (m, 1)
y = np.matrix([range(2,5)]).T

# Calculate w*
w_ast = np.matrix([])

In [ ]:
w_ast

**Function definition**

In [ ]:
def polyfit_inv_normal_eq(x: np.matrix,
                          y: np.matrix,
                          n: int):
    """
    Fits a polynomial of degree n to a sets of samples x and y.
    The polynomial minimizes the sum of squared errors (least squares)
    solving the normal equations.

    :param x: m x 1, points in the x axis where the function is known
    :param y: m x 1, known values of the function at positions in x
    :param n: number of coefficients (degree of the polynomial + 1)

    :return w: n x 1, vector of polynomial coefficients
    :return phi: m x n, data matrix
    """
    # TODO: create phi
    phi = np.matrix([])
    
    # TODO: create w*
    w_ast = np.matrix([])
    
    return phi, w_ast

In [ ]:
polyfit_inv_normal_eq(x, y, num_coefficients)

#### 2. Complete the function polyfit_main, to verify that the w&ast; returned by polyfit_inv_normal_eq satisfies that the residue r = y − &phi;w&ast; is orthogonal to Im&phi;, i.e. r ⊥ Im&phi;. Hint: Im&phi; is the space generated by the columns of &phi;.

**Generate random data**

We will generate an artificial data set. We do it by generating random
samples from the following polynomial:
 
p(x) = w<sub>0</sub> + w<sub>1</sub>\*x + w<sub>2</sub>\*x<sup>2</sup> + w<sub>3</sub>\*x<sup>3</sup> 
 
We generate some x<sub>i</sub>, with their corresponding y<sub>i</sub>, and then add random noise to the y<sub>i</sub> to simulate errors in the measurement. 

In [ ]:
#Bigger example
m=1000
#Set seed value. Use same whenever calling a random number generator
seed_value = None  
np.random.seed(seed_value)
x = np.matrix(np.random.uniform(-13,13,(m,1)))
y = np.power(x,range(0,num_coefficients))@np.random.uniform(size=(num_coefficients,1)) + np.random.uniform(-1,1,size=(m,1))

Build up a graphic that shows p(x)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(x, y, '.')

**Calculate the residue**

**Demostrate r ⊥ Im&phi;**

To demonstrate that r ⊥ Im&phi; we need to check that the scalar product of both matrixes is 0

In [ ]:
residue = np.matrix([])
scalar_product = np.matrix([])
scalar_value = -1
print("Scalar product matrix: \n\n{}\n".format(scalar_product))
print("Scalar product solution: {}".format(scalar_value))

**Function definition**

In [ ]:
def polyfit_main(n_data_samples: int = 100,
                 polynomial_size: int = 4,
                 lower_data_value: int = -10,
                 higher_data_value: int = 10):
    """
    Fits a polynomial of degree n to a set of samples.

    :param n_data_samples: number of samples to be created
    :param polynomial_size: polynomial size (number of elements in array)
    :param lower_data_value: lowest value in data
    :param higher_data_value: highest value in data

    :return:
    """
    # TODO: create x
    x = np.matrix([])

    # TODO: create y to fit p(x) = w0 + w1 * x + w2 * x^2 + w3 * x ^ 3 + ... + wn * x ^ n --> n = polynomial_size
    y = np.matrix([])

    # Plot the graphic
    plt.figure()
    plt.plot(x, y, '.')
    plt.show()

    # TODO: Calculate w*
    w_ast = np.matrix([])

    # TODO: Calculate the residue and demostrate it is orthogonal to Phi
    residue = np.matrix([])
    scalar_product = np.matrix([])
    scalar_value = -1
    print("Scalar product matrix: \n\n{}\n".format(scalar_product))
    print("Scalar product solution: {}".format(scalar_value))

In [ ]:
polyfit_main()

## 1.2 Minimization with the SVD

#### The pseudo-inverse

##### 1) Generate a 5 × 3 random matrix, A. Compute its SVD A = USV<sup>T</sup> using the command svd. Verify that A = USV<sup>T</sup>.

**Generate A**

In [ ]:
A = np.matrix([])
print(A)

**Compute its SVD**

Numpy provides a function to compute the SVD of a given matrix. It is accesible from its *linalg* object, and it will return U, S and V<sup>T</sup>. Moreover, S is not a rectangular diagonal matrix as expected, so in order to perform real operations with it you must use *diag(S)*

In [ ]:
U, S, Vt = np.matrix([]), np.matrix([]), np.matrix([])
print("U: \n\n{}\n\nS: \n\n{}\n\nVt: \n\n{}".format(U, S, Vt))

**Verify that A = USV<sup>T</sup>**

In [ ]:
# Calculate A
A_recovered = np.matrix([])

# Check if calculated A is equal to, or really close to, the original A
print("Is verified that A = U * S * transpose(V): {}".format(np.allclose(A, A_recovered)))

**Verify that U and V are orthogonal matrixes**

To verify that we will use the following consideration: If U and V are orthogonal, then (U * V)<sup>-1</sup> = (U * V)<sup>T</sup> 

In [ ]:
Q = np.matrix([]) 
B = np.matrix([])
C = np.matrix([])
print("Q\n\n{}\n\nB\n\n{}\n\nC\n\n{}\n\nIs U ⊥ V? {}".format(Q, B, C, np.allclose(B, C)))

##### 2) Compute the pseudo-inverse of S, S†. Use it to compute the pseudo-inverse of A, A†. Verify that A†A is the identity matrix of size 3. What happens with AA†?

**Compute the pseudo-inverse of S**

In [ ]:
S_pinv = np.matrix([])
print(S_pinv)

**Compute the pseudo-inverse of A by using the pseudo-inverse of S**

In [ ]:
A_pinv = np.matrix([])
print(A_pinv)

**Verify that A†A is the identity matrix of size 3**

In [ ]:
I_ = np.matrix([])
print("A†A:\n\n{}\n".format(I_))
print("A†A is the identity matrix of size 3?: {}".format(np.allclose(I_, np.identity(n = len(I_)))))

#### Solving least squares with the pseudo-inverse

#### 5. Complete the function polyfit_svd_normal_eq for computing w\*

$$w^{*} = \Phi^{\dagger}y = VS^{\dagger}U^{T}y$$

**1) Calculate Phi**

In [ ]:
# Define x --> (m, 1)
x = np.matrix([]).T

# Define the number of coefficients to calculate (components of w --> w = (w0, w1, ..., wn))
num_coefficients = 4

# Calculate phi
phi = np.matrix([])
print("Phi: \n\n{}".format(phi))

**2) Calculate its SVD**

In [ ]:
U, S, Vt = np.matrix([]), np.matrix([]), np.matrix([])
print("U: \n\n{}\n\nS: \n\n{}\n\nVt: \n\n{}".format(U, S, Vt))

**3) Compute the pseudo-inverse of S**

In [ ]:
S_pinv = np.matrix([])
print("Pseudo-inverse of S: \n\n{}".format(S_pinv))

**4) Calculate w\***

In [ ]:
# Define y --> (m, 1)
y = np.matrix([]).T

# Compute w*
w_ast = np.matrix([])
print("W*\n\n{}\n".format(w_ast))
print("Is current w* equal to polyfit_inv_normal_eq output? {}".format(np.allclose(w_ast, polyfit_inv_normal_eq(x, y, num_coefficients)[1])))

**Function definition**

In [ ]:
def polyfit_svd_normal_eq(x: np.matrix,
                          y: np.matrix,
                          n: int):
    """
    Fits a polynomial of degree n to a sets of samples x and y. The polynomial minimizes the sum 
    of squared errors (least squares) using the pseudoinverse of data matrix.
    
    :param x: m x 1, points in the x axis where the function is known
    :param y: m x 1, known values of the function at positions in x
    :param n: number of coefficients (degree of the polynomial + 1)
    
    :return w_ast: n x 1, vector of polynomial coefficients
    :return phi_pinv: n x m, pseudo-inverse of Phi
    :return phi: m x n, data matrix
    """
    # TODO: Calculate phi
    phi = np.matrix([])
    
    # TODO: Calculate its SVD
    U, S, Vt = np.matrix([]), np.matrix([]), np.matrix([])
    
    # TODO: Calculate the pseudo-inverse of S
    S_pinv = np.matrix([])
    
    # TODO: Calculate w*
    phi_pinv = np.matrix([])
    w_ast = np.matrix([])
    
    return w_ast, phi_pinv, phi

# 2. Principal Components Analysis

#### 1. Complete the function pca_prin_dir for computing the p first principal directions via the eigenvectors of X<sup>T</sup>X.


**1) Center X**

In [ ]:
# Define x --> (m, n)
x = np.matrix([])
print("X: \n\n{}\n".format(x))

# Calculate its mean
mu = np.matrix([])
print("Mean: {}\n".format(mu))

# Center X
x = x - mu
print("X centered: \n\n{}\n".format(x))

**2) Calculate the empirical covariance matrix**

In [ ]:
C = np.matrix([])
print("Covariance matrix: \n\n{}".format(C))

**3) Compute p-first principal directions**

In [ ]:
# Define the number of p-directions to be retrieved
p = 1

# Compute the principal directions, sort them by lambda values (eigen values) and return only 
# the p-first values
eigen_values, eigen_vectors = np.matrix([]), np.matrix([])

In [ ]:
print("Eigen values: \n\n{}\n\nEigen vectors: \n\n{}".format(eigen_values, eigen_vectors))

**Function definition**

In [ ]:
def pca_prin_dir(x: np.matrix,
                 p: int):
    """
    Computes the principal directions, variances and mean of vectors in x. 
    x is a row data matrix: its rows are vectors xi. The principal directions are given 
    by eigenvectors of the empirical covariance matrix, x'*x.
        
    :param x: row data matrix, m x n
    :param p: number of principal directions
    
    :return eigen_vectors: principal direction matrix (each column is a PD) n x p
    :return eigen_values: eigenvalue diagonal matrix p x p
    :return mu: mean
    """
    # TODO: Calculate the mean of the incoming data and center x
    mu = np.matrix([])
    
    # TODO: Calculate the empirical covariance matrix
    C = np.matrix([])
    
    # TODO: Compute the principal directions and sort them by lambda values (eigen values) and return only 
    # the p-first values
    eigen_values, eigen_vectors = np.matrix([]), np.matrix([])

    return eigen_vectors, eigen_values, mu

In [ ]:
pca_prin_dir(x, p)

#### 2. Complete the function pca_prin_comp for computing the p first principal components [<x, v<sub>i</sub>>]<sub>i=1, ..., p</sub> of a point x


- Equation to be used: z<sub>p</sub> = V<sub>p</sub><sup>T</sup>x --> z<sub>i</sub> = [<x<sub>i</sub>-mu,v<sub>1</sub>> <x<sub>i</sub>-mu,v<sub>2</sub>> ... <x<sub>i</sub>-mu,v<sub>p</sub>> ]


- Where: x = $\hat{x}$ - &mu;

**1) Center X**

In [ ]:
# Define x --> (m, 1)same as before
x = np.matrix([])
print("X: \n\n{}\n".format(x))

# Calculate its mean
mu = np.matrix([])
print("Mean: {}\n".format(mu))

# Center X
x = np.matrix([])
print("X centered: \n\n{}\n".format(x))

**2) Project x over basis elements to compute principal components**

In [ ]:
z_p = np.matrix([])

In [ ]:
z_p

**Function definition**

In [ ]:
def pca_prin_comp(x: np.matrix,
                  eigen_vectors: np.matrix,
                  mu: np.matrix = None):
    """
    Computes the principal components of vectors in x. x is a row data matrix: its rows are vectors xi. 
    The principal components zi of a vector xi are given by the projection over the principal directions: 

                            zi = [ <xi-mu,v1> <xi-mu,v2> ... <xi-mu,vp> ]

    :param x: row data matrix, m x n
    :param eigen_vectors: principal direction matrix (each column is a PD) n x p

    :return z : m x p, principal component matrix. Row i contains the princpal components of xi
    """
    # TODO: Calculate the raw data mean along row axis and center x

    # Project x over basis elements to compute principal components
    z = np.matrix([])  
    
    return mu, z

In [ ]:
pca_prin_comp(x = x,
              eigen_vectors = eigen_vectors)

#### 3. Complete the function pca_reconstruct for reconstructing a point x from its principal components.


- Equation to be used: $\hat{x}$ = z<sub>p</sub>V<sub>p</sub><sup>T</sup>
- Where: x = $\hat{x}$ + &mu;


In [ ]:
# Calculate x_hat
x_hat = np.matrix([])

# Add the mean to recover the real matrix values
x_ = np.matrix([])

**Function definition**

In [ ]:
def pca_reconstruct(z: np.matrix([]),
                    eigen_vectors: np.matrix([]),
                    mean: float):
    """
    Given a set of vectors z(i,:) (rows of matrix z) expressed in principal components, this function computes the x(i,:), 'inverting' the PCA change of coordinates. 
    It goes from the low dimensional PCA representation to the high dimensional vectors.

    :param z: m x p, principal component matrix (row-wise). 
    :param V: principal direction matrix (each column is a PD) n x p
    :param mu: mean

    :return x: row data matrix, m x n. Row x(i,:) is the high dimensional reconstruction from z(i,:)
    """
    # Calculate x_hat
    x_hat = np.matrix([])

    # Add the mean to recover the real matrix values
    x_ = np.matrix([])
    
    return x_

In [ ]:
# z_p is calculated in the step before
z_p = np.matrix([]) # --> Comment/delete this line when the above one is run
pca_reconstruct(z = z_p, eigen_vectors = eigen_vectors, mean = mu)

#### 4. For the flat ellipsoid dataset (provided together with the code) run the following lines. Four figures will open. Read the code and explain what each figure is showing. Answer the questions asked in the code.

In [ ]:
# Install a widget that will allow you to build up interactive plots
# !conda install -c conda-forge ipympl -y

# If using JupyterLab
# !conda install -c conda-forge nodejs -y
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-matplotlib

# After run this lines, close the jupyter session and restore it

**1) Read the CSV file which contains the data and plot them**

In [ ]:
#pip install mpld3

In [ ]:
#pip install ipympl

In [ ]:
# Import Pandas --> see https://pandas.pydata.org/ 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import mpld3
import matplotlib.cm as cm
%matplotlib widget

In [ ]:
# Read the file
data = pd.read_csv("../data/point_cloud_r3.csv", sep = ",", header=None, names = ["x", "y", "z"])
data.head()

In [ ]:
# Plot the data
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d', )
ax.scatter(data['x'], data['y'], data['z'], s=10, alpha=0.9, edgecolors='w', depthshade = False)
plt.xticks(list(range(int(data['x'].min()), int(data['x'].max()), 1)))
plt.yticks(list(range(int(data['y'].min()), int(data['y'].max()), 1)))
ax.set_zticks(list(range(int(data['z'].min()), int(data['z'].max()), 1)))

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

**2) Compute principal all principal directions and the correspoding eigenvalues**

**NOTE:** in this example we are going to do some visualization of the principal
components in IR^3. For that we will compute ALL principal directions and ALL
principal components. Please note that this is not the typical way in which
PCA is used.  In a normal application of dimensionality reduction we would
compute only a few principal directions and principal components.

In [ ]:
# Define the number of principal directions 
p = len(data.columns)

# Calculate the PCA principal directions for the given data
eigen_vectors, eigen_values, mu = pca_prin_dir(data.values, p)
print("Eigen values: \n\n{}\n\nEigen vectors: \n\n{}".format(eigen_values, eigen_vectors))

#### Answer the following questions:
**a) What can you tell from the eigenvalues about the geometry of the original point set?**

Answer: 

**b) Which is the number of components you would keep to hold at least the 95% of the variance?**

In order to answer this question, we must represent the explained variance in terms of participation. 

In [ ]:
# Calculate the participation in terms of %
tot = sum(eigen_values)
var_exp = [(i / tot)*100 for i in sorted(eigen_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

# Plot the graphic
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(6, 4))

    plt.bar(range(len(eigen_values)), var_exp, alpha=0.5, align='center',
            label='individual explained variance')
    plt.step(range(len(eigen_values)), cum_var_exp, where='mid',
             label='cumulative explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

Answer: 

**c) How large would be the mean projection error we would commit by keeping only the first principal component?**

In theory, MSE and eigenvalues are related by the following equation:

\begin{equation*}
\frac{1}{m}\sum^{n}_{i=1}\|x_{i}-P_{V_{p}}(x_{i})\|^{2}= \sum^{n}_{j=p+1}\lambda_{j}
\end{equation*}

Therefore, we will check that in both cases we achieve the same result.

In [ ]:
# Define the number of components to be kept
components = 2
components_array = list(range(components))

# Calculate the projection of x over the eigenvectors basis by only keeping the first component
mu, z = pca_prin_comp(x = data.values,
                      eigen_vectors = eigen_vectors)
projection = pca_reconstruct(z = z[:, components_array], eigen_vectors = eigen_vectors[:, components_array], mean = mu)

# Calculate the MSE
print("MSE for the first {} components: {}".format(components, np.power(data.values - projection, 2).sum() / (len(data) - 1)))

**Function definition**

In [ ]:
def calculate_projection_mse(x: np.matrix,
                             eigen_vectors: np.matrix,
                             z: np.matrix,
                             p: np.matrix,
                             mean: float):
    """
    Calculates the projection MSE
    
    :param x: row data matrix, m x n
    :param eigen_vectors: principal direction matrix (each column is a PD) n x p
    :param z: m x p, principal component matrix. Row i contains the princpal components of xi    
    :param p: number of components to be used when projecting
    :param mu: mean
    
    :return:
    """
    # Define the number of components to be kept
    components_array = list(range(p))

    # Calculate the projection of x over the eigenvectors basis by only keeping the first component
    mu, z = pca_prin_comp(x = x,
                          eigen_vectors = eigen_vectors)
    projection = pca_reconstruct(z = z[:, components_array], eigen_vectors = eigen_vectors[:, components_array], mean = mean)

    # Calculate the MSE
    mse = np.power(x - projection, 2).sum() / (len(x) - 1)
    
    return mse

In [ ]:
components = 1
print("MSE for the first {} components: {}".format(components, calculate_projection_mse(x = data.values,
                                                                                        eigen_vectors = eigen_vectors,
                                                                                        z = z,
                                                                                        p = components,
                                                                                        mean = mu)))

**d) And if we keep the first and the second?**

In [ ]:
components = 2
print("MSE for the first {} components: {}".format(components, calculate_projection_mse(x = data.values,
                                                                                        eigen_vectors = eigen_vectors,
                                                                                        z = z,
                                                                                        p = components,
                                                                                        mean = mu)))

**3) Now plot the point cloud in 3D. Do three plots, in each of them will color the points with the value of one principal component (use the provided function plot_3d_and_components). Also plot the principal directions as vectors at the origin. The length of these vectors is proportional to the corresponding eigenvalue lambda.**

In [ ]:
def plot_3d_and_components(xs: np.matrix,
                           ys: np.matrix,
                           zs: np.matrix,
                           means: np.matrix,
                           principal_components: np.matrix,
                           p: int,
                           eigen_vectors: np.matrix = None,
                           color_map: list = []):
    """
    Plots a 3D graph and the PCA components
    
    :param xs: x-axis data
    :param ys: y-axis data
    :param zs: z-axis data
    :param means: means per axis
    :param eigen_vectors: eigenvectors
    :param principal_components: principal component matrix
    :param p: number of components to use
    :param color_map: color map to be used when representing
    """
    # Import libraries
    import matplotlib
    import inflect

    # Calculate the maximum and minimum values
    components = list(range(p))
    minimas = principal_components[:, components].min(axis = 0)
    maximas = principal_components[:, components].max(axis = 0)
    
    # Build up a object to convert from number to ordinal 
    p = inflect.engine()

    # Build up the colors map and plot the figures
    fig = plt.figure(figsize=(10, 8))
    xticks = list(range(int(xs.min()), int(xs.max()), 1))
    yticks = list(range(int(ys.min()), int(ys.max()), 1))
    zticks = list(range(int(zs.min()), int(zs.max()), 1))
    for component in components:
        norm = matplotlib.colors.Normalize(vmin=minimas[component], vmax=maximas[component], clip=True)
        mapper = cm.ScalarMappable(norm=norm, cmap=color_map[component])
        color = [mapper.to_rgba(v)[0] for v in z[:, [component]]]
        
        # Plot the data
        ax = fig.add_subplot(1, len(components), component + 1, projection='3d')
        ax.scatter(xs, ys, zs, s=10, c = color, alpha=0.9, edgecolors='w', depthshade = False)
        if eigen_vectors is not None:
            for v in eigen_vectors.T:
                ax.plot([means[0] - v[0], v[0] + means[0]], 
                        [means[1] - v[1], v[1] + means[1]],
                        [means[2] - v[2], v[2] + means[2]], 
                        linewidth = 3,
                        color = "darkblue")
        ax.set_xticks(xticks)
        ax.set_yticks(yticks)
        ax.set_zticks(zticks)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title("Principal directions and {} principal component (color).".format(p.ordinal(component + 1)), fontdict = {"fontsize": 6})
    plt.show()

In [ ]:
plot_3d_and_components(xs = data.x,
                       ys = data.y,
                       zs = data.z,
                       means = mu,
                       eigen_vectors = eigen_vectors * eigen_values * 0.5,
                       principal_components = z,
                       p = 3,
                       color_map = [cm.gnuplot, cm.gnuplot, cm.gnuplot])

**4) Now let's do some dimensionality reduction. For that, we keep some of theprincipal components z.**

**a) Keep the first two principal components, project the points into a two dimensional vector space and calculate the projection error.**

In [ ]:
def plot_3d_and_projections(xs: np.matrix,
                            ys: np.matrix,
                            zs: np.matrix,
                            xp: np.matrix,
                            yp: np.matrix,
                            zp: np.matrix,
                            title: str = ""):
    """
    Plots a 3D graph and the PCA components
    
    :param xs: x-axis data
    :param ys: y-axis data
    :param zs: z-axis data
    :param xp: x-axis projection data
    :param yp: y-axis projection data
    :param zp: z-axis projection data
    """
    # Import libraries
    import matplotlib

    # Build up the colors map and plot the figures
    fig = plt.figure(figsize=(10, 8))
    xticks = list(range(int(xs.min()), int(xs.max()), 1))
    yticks = list(range(int(ys.min()), int(ys.max()), 1))
    zticks = list(range(int(zs.min()), int(zs.max()), 1))
    
    # Plot the data
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    ax.scatter(xs, ys, zs, s=10, c = "darkblue", alpha=0.9, edgecolors='w', depthshade = False)
    ax.scatter(xp, yp, zp, s=10, c = "red", alpha=0.9, edgecolors='w', depthshade = False)
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    ax.set_zticks(zticks)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(title, fontdict = {"fontsize": 6})
    plt.show()

First and second components

In [ ]:
# Calculate the projection of x over the eigenvectors basis by only keeping the first and second components
components_array = [0, 1]
mu, z = pca_prin_comp(x = data.values,
                      eigen_vectors = eigen_vectors)
projection = pca_reconstruct(z = z[:, components_array], eigen_vectors = eigen_vectors[:, components_array], mean = mu)
error_projection_fs = np.sqrt(np.power(data.values - projection, 2).sum(axis = 1))

# Plot the projection
plot_3d_and_projections(xs = data.x,
                        ys = data.y,
                        zs = data.z,
                        xp = projection[:, 0],
                        yp = projection[:, 1],
                        zp = projection[:, 2],
                        title = "Real data versus projection over the first and second components")

**b) Keep the first and third principal components, project the points into a two dimensional vector space and calculate the projection error.**

In [ ]:
# Calculate the projection of x over the eigenvectors basis by only keeping the first and second components
components_array = [0, 2]
mu, z = pca_prin_comp(x = data.values,
                      eigen_vectors = eigen_vectors)
projection = pca_reconstruct(z = z[:, components_array], eigen_vectors = eigen_vectors[:, components_array], mean = mu)
error_projection_ft = np.sqrt(np.power(data.values - projection, 2).sum(axis = 1))

# Plot the projection
plot_3d_and_projections(xs = data.x,
                        ys = data.y,
                        zs = data.z,
                        xp = projection[:, 0],
                        yp = projection[:, 1],
                        zp = projection[:, 2],
                        title = "Real data versus projection over the first and third components")

**c) Which 2D subspace approximates better the original points?**

Answer: 

**5) Plot the low dimensional representative z in IR^2 and color them with the projection error. Look at the colorbars to compare the error in both plots.**

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(10, 8), ncols=2)
sc = ax1.scatter(z[:, 0] , z[:, 1], s=10, c = error_projection_fs, alpha=0.9, edgecolors='w')
plt.colorbar(sc, ax = ax1)
ax1.set_title("First and second Principal Components")
sc = ax2.scatter(z[:, 0] , z[:, 2], s=10, c = error_projection_ft, alpha=0.9, edgecolors='w')
plt.colorbar(sc, ax = ax2)
ax2.set_title("First and third Principal Components")
plt.show()

#### 5. This is the first script of the face recognition application. The idea here is to run a very simple face recognition algorithm, but using PCA to reduce the dimensionality of the vectors. The original vectors are images of 211 x 229. If we consider the images as vectors in IR^n (each pixel is a component of the vector), we have that n = 221x229 = 48319. Using PCA, we will reduce the dimensionality to ~ 50, 1000 times smaller!!!

In [ ]:
# In case needed:
#!conda install -c anaconda pillow -y

In [ ]:
from PIL import Image
import os

**1) Load faces from image files** 

Images are in data/faces folder. We will load them and store them as rows in a data matrix

In [ ]:
# List all images and read them. Once read, reshape them into (1, W x H)
faces_dir = os.path.abspath("../data/faces")
faces_files = [file_ for file_ in os.listdir(faces_dir) if file_.endswith("png")]
faces_shape = np.array(Image.open(os.path.join(faces_dir, faces_files[0]))).shape
faces = np.empty((len(faces_files), faces_shape[0] * faces_shape[1])) 
for index, img in enumerate(faces_files):
    im_frame = Image.open(os.path.join(faces_dir, img))
    np_frame = np.array(im_frame)
    faces[index] = np_frame.reshape((1, faces_shape[0] * faces_shape[1]))

**2) Split data randomly into a train set (tr), and a test set (ts)**

Leave a part of the data set to test. The test data set are faces we will try to recognize.

The sizes will be 80% for training and 20% for testing

In [ ]:
import random

In [ ]:
# Get the training and testing faces indexes
seed_value = None  
np.random.seed(seed_value)
test_faces_idx = random.sample(list(range(len(faces))), k = int(len(faces) * 0.205))
train_faces_idx = list(set(list(range(len(faces)))).difference(set(test_faces_idx)))

# Build up the training and testing faces
test_faces = faces[test_faces_idx, :]
train_faces = faces[train_faces_idx, :]

In [ ]:
len(train_faces)

**3) Train PCA model**

Now we will compute the principal directions using the training set. 

**Note:** the covariance matrix is too large to fit in memory. We can circumvent this problem using the SVD of x. Recall that the eigenvectors of x'\*x correspond to the right singular vectors (columns of V):

x = U\*S\*V'

x'\*x = V\*S'\*U'\*U\*S\*V' = V\*S'\*S\*V' 

However, this does not work either. We have the same problem! The size of V does not fit in memory. This is not surprising, both C and V are n x n matrices. 

We can solve this problem with the economic size SVD. Note that n - m columns of V are multiplied by the zeros of S, in the product U\*S\*V'. This means that we can remove this columns. The economic size SVD is the following: 

x = U\*S0\*V0'

where S0 is m x m and V0 is n x m. The same can be done with the U matrix in the case m > n.

We want to reduce V, not U. We can solve this problem very easily, by computing the svd of x'. It can be shown that if

x = U\*S\*V'

is the SVD of x, then the svd of x' is given by

x' = V\*S'\*U

The same happens with the economic size svd. Thus we will compute V0 via the economic svd of x'.

In [ ]:
# Define P
p = len(train_faces)

# Calculate the SVD of x' (x = x - mean(x))
U, S, Vt = np.linalg.svd((train_faces - train_faces.mean(axis = 0)).T, full_matrices=False)

# Keep only the first p 
V  = U[:, :p]
S = np.diag(S)[:p, :p]

# The eigenvalues of x'*x are the square of the singular values
S = np.power(S, 2)

**4) Visualize the PCA results**

In [ ]:
# Calculate the participation in terms of %
tot = sum(np.diag(S))
var_exp = [(i / tot)*100 for i in sorted(np.diag(S), reverse=True)]
cum_var_exp = np.cumsum(var_exp)

# Plot the graph
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(8, 6))
    plt.bar(range(len(np.diag(S))), var_exp, alpha=0.5, align='center',
            label='individual explained variance')
    plt.step(range(len(np.diag(S))), cum_var_exp, where='mid',
             label='cumulative explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

As you can see in graph above, by using the first 50 principal components we can keep almost the 90% of the total variance. Let's take a look at the *mean face*

In [ ]:
# Define the number of principal components to keep
p = 50

# Keep only the first p principal directions
V_p = V[:, :p]
S_p = S[:p, :p]

# Plot the mean face
plt.figure()
plt.imshow(train_faces.mean(axis = 0).reshape(faces_shape), cmap = "gray")
plt.show()

And now build a big image with the 6 first principal directions


In [ ]:
# Build up the images matrix
num_rows = 2
num_columns = 3
tmp = np.empty(shape = (faces_shape[0] * num_rows, faces_shape[1] * num_columns))
component_index = 0
for row in range(num_rows):
    for column in range(num_columns):
        tmp[faces_shape[0] * row: faces_shape[0] * (row + 1), faces_shape[1] * column: faces_shape[1] * (column + 1)] = (S_p[component_index, component_index] * V_p[:, component_index]).reshape(faces_shape)
        component_index += 1

# Plot the images
plt.figure()
plt.imshow(tmp, cmap = "gray")
plt.show()

**5) project a test face over principal directions**

In [ ]:
# Keep one of the test faces
x = test_faces[0, :]

# Build up the principal components
_, z = pca_prin_comp(x = x, eigen_vectors = V_p, mu = train_faces.mean(axis = 0)); 

# Reconstruct the face
x_proj = pca_reconstruct(z = z, eigen_vectors = V_p, mean = train_faces.mean(axis = 0))

# Show both faces
fig, (ax1, ax2) = plt.subplots(figsize=(8, 6), ncols=2)
ax1.imshow(x.reshape(faces_shape), cmap = "gray")
ax1.set_title("Test image")
ax2.imshow(x_proj.reshape(faces_shape), cmap = "gray")
ax2.set_title("Projected image")
plt.show()

**6) Recognize faces**

This a very simple face recognition algorithm: it is based on comparing the PCA coordinages of a query face with those of the training set.

In [ ]:
# Import a library to simulate time steps
import time
%matplotlib inline

# Define the number of principal components to keep
p = 50

# Keep only the first p principal directions
V_p = V[:, :p]
S_p = S[:p, :p]

# Compute principal components for training faces
_, train_z = pca_prin_comp(x = train_faces, eigen_vectors = V_p, mu = train_faces.mean(axis = 0))

# Compute principal components for test faces
_, test_z = pca_prin_comp(x = test_faces, eigen_vectors = V_p, mu = train_faces.mean(axis = 0))

# Classify
nn_idx = []
for i in range(len(test_faces)):
    d = np.linalg.norm(np.tile(test_z[[i], :], (train_z.shape[0], 1)).T - train_z.T, axis = 0).T
    nn_idx.append(int(d.argmin()))

# Visualize classification results
plt.figure()
plt.ion()
for i in range(len(test_faces)):
    print("Showing image {} of {}".format(i + 1, len(test_faces)))
    plt.subplot(1, 2, 1).imshow(test_faces[i, :].reshape(faces_shape), cmap = "gray")
    ax1.set_title("Test image")
    plt.subplot(1, 2, 2).imshow(train_faces[nn_idx[i], :].reshape(faces_shape), cmap = "gray")
    ax2.set_title("Training image")
    plt.show()
    plt.pause(1)